In [ ]:
# !pip install taipy
# !pip install python-airtable

In [1]:
from taipy import Gui
import requests
import pandas as pd
import datetime
import numpy as np
import random
from datetime import datetime
from datetime import timedelta
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import squarify
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import seaborn as sns
from airtable import Airtable
import os
import re

#### Airtable - fetch data

In [2]:
# def connect_airtable():
#     api_key = 'patngX23FWWWnl2iH.e5c8bc51522c7d6db26f4668e80775aa7f344107ec6a1204a5b388427a6e1da7'

#     url = 'https://api.airtable.com/v0/appfjmrH6l5m4Z82K/tbly4c5vUFLAP91ga'

#     headers = {'Authorization': f'Bearer {api_key}'}

#     #request to airtable
#     response = requests.get(url, headers=headers)
#     data = response.json()
#     records = []
    
#     for record in data['records']:
#         fields = record['fields']
#         records.append(fields)
#     pd.set_option('display.max_rows', None)
#     pd.set_option('display.max_columns', None)
#     pd.set_option('display.width', 9999)
#     df = pd.DataFrame(records)
#     return df
# # 

In [3]:
api_key = 'patngX23FWWWnl2iH.e5c8bc51522c7d6db26f4668e80775aa7f344107ec6a1204a5b388427a6e1da7'
base_id = 'appfjmrH6l5m4Z82K'
table_name_mapping = {
    'tblM34kdan10sEZrK': 'items',
    'tblSYZrSYVQJcvNkR': 'customers',
    'tblkGA9TFjiGyiqTN': 'payments',
    'tblm5kBGMQg08mIej': 'suppliers',
    'tblFuLuXFBtrqHbe3': 'catalog',
    'tbly4c5vUFLAP91ga': 'orders'
}


In [4]:
def fetch_all_airtable_data(api_key, base_id, table_name_mapping):
    all_dataframes = {}
    for table_id, table_name in table_name_mapping.items():
        airtable = Airtable(base_id, table_id, api_key)
        records = []
        for record in airtable.get_all():
            records.append(record['fields'])
        df = pd.DataFrame.from_records(records)
        all_dataframes[table_name] = df
        df.to_csv(f'{table_name}.csv', index=False)
    return all_dataframes

In [5]:
def clean_fetched():
    fetch_all_airtable_data(api_key, base_id, table_name_mapping)
    pd.set_option('display.max_colwidth', None)
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)
    orders = pd.read_csv('orders.csv')
    pay = pd.read_csv('payments.csv')
    # Use a regular expression to extract digits or NaN
    orders['Num (from Customer)'] = orders['Num (from Customer)'].str.extract(r"(\d+|\d+\.\d+|\d+\.\d+e\+\d+|\d+\.\d+e-\d+|\d+e\+\d+|\d+e-\d+|\d+\.\d+e-\d+|\d+\.\d+e\+\d+|NaN)", expand=False)
    # Convert the 'Extracted_Num' column to numeric (handles NaN)
    orders['Num (from Customer)'] = pd.to_numeric(orders['Num (from Customer)'], errors='coerce')
    orders['Num (from Customer)'] = orders['Num (from Customer)'].astype(pd.Int64Dtype(), errors='ignore')
    orders['Order num'] =pd.to_numeric(orders['Order num'], errors='coerce').astype('float64')
    orders.dropna(subset=['Num (from Customer)'], inplace=True)
    # Use a regular expression to extract digits or NaN
    pay['Num (from Customer)'] = pay['Num (from Customer)'].str.extract(r"(\d+|\d+\.\d+|\d+\.\d+e\+\d+|\d+\.\d+e-\d+|\d+e\+\d+|\d+e-\d+|\d+\.\d+e-\d+|\d+\.\d+e\+\d+|NaN)", expand=False)
    # Convert the 'Extracted_Num' column to numeric (handles NaN)
    pay['Num (from Customer)'] = pd.to_numeric(pay['Num (from Customer)'], errors='coerce')
    pay['Num (from Customer)'] = pay['Num (from Customer)'].astype(pd.Int64Dtype(), errors='ignore')
    pay.sort_values(by=['Num (from Customer)', 'Order # (from Orders)', 'Requested on'], ascending=[True, True, True], inplace=True)
    pay.drop_duplicates(subset=['Num (from Customer)', 'Order # (from Orders)'], keep='last', inplace=True)
    pay['Order # (from Orders)'] =pd.to_numeric(pay['Order # (from Orders)'], errors='coerce').astype('float64')
    
    orders.sort_values(by=['Num (from Customer)', '#', 'Order num','Invoice #'], ascending=[True, True, True, True], inplace=True)
    
    dfa = pd.merge(orders, pay, how='left', left_on=['Num (from Customer)', 'Order num'], right_on=['Num (from Customer)', 'Order # (from Orders)'])
    
    cust = pd.read_csv('customers.csv')
    cust['Num'] = pd.to_numeric(cust['Num'], errors='coerce').astype('Int64')
    cust.dropna(subset=['Name'], inplace=True)
    cust.dropna(subset=['Num'], inplace=True)
    dfa2 = pd.merge(dfa, cust, how='outer', left_on='Num (from Customer)', right_on='Num')
    dfa2['Num (from Customer)'].fillna(dfa2['Num'], inplace=True)
    dfa2['Name (from Customer)'].fillna(dfa2['Name'], inplace=True)
    dfa2['Project name'].fillna(dfa2['Project name (from Orders)'], inplace=True)
    dfa2['Project name (from Orders)'].fillna(dfa2['Project name'], inplace=True)
    dfa2['Invoicing address (from Customer)_x'].fillna(dfa2['Invoicing address'], inplace=True)
    dfa2.drop_duplicates(subset=['Num (from Customer)', 'Order num', 'Project name'], keep='last', inplace=True)
    dfa2.drop_duplicates(subset=['Num (from Customer)','Email (from Manager)','Order total','Created: project name','Date quote sent'], keep='last', inplace=True)
    
    to_drop = ['Manager_x','Today_x','Teams (from Manager)',
           'Team Emails (from Managers) (from Teams) (from Manager)_x',
          'Customer_x','DELETE invoice total','Year','Invoicing contact (from Customer)_x',
           'Invoice button','Items Table_Id','Customer Table_Id',
           'Open URL','Order items Table_Id','Payments Table_Id',
          'Order to supplier Table_Id','Create quote','Payments_button','Paper track',
           'Button quote','Empty order items alert','Check order number',
          'Prod and ship','Ship addresses missing (from Customer)','Create BAT',
          'Check signed BAT','Create mold invoice','Create cost PDF','Alert mold invoice sent',
          'Alert MP invoice payment','Create supplier contract', 'Alert sample approval',
          'Manage invoices', 'Update Customer info','Sample order','MP order',
          '#','Invoice #','Customer name in text', 
            'Quantity (from Shipments) (from Order items)','Remains to be shipped (from Order items)','Invoicing address','Invoice Ref',
            'Customer name in text','Manage BAT','Total on invoice', 'Balance due on invoice',
            'Email (from Invoicing contact) (from Customer)_x','Addresses missing (from Customer)',
            'Today plus 7','Days RS', 'Days since R','Paid for MP downpayment',
            'Paid for MP','Percentage paid','Order items','Total (from Order items)',
            'On invoice (from Order items)','Order unit cost ¥ (from Order items)',
           'Cost check (from Order items)','Price check (from Order items)','Total price on invoice (from Order items)',
            'Total price mold invoice (from Order items)','Picture (from Order items)',
            'Customer contact','Quote PDF','Project type','Payments_x',
            'Amount paid (from Invoices)','Status of payment (from Payments)',
            'Picture for this order (from Order items)','Picture for this order (from Order items) 2',
                        'Status mold payment (from Payments)','Status MP downpayment (from Payments)',
                        'Order photo (from Order items)','Material (from Order items)','Order type',
           'Payments to suppliers','Payment date (from Payments to suppliers)','Supplier (from Payments to suppliers)',
                        'Percentage of downpayment','Balance due time','BAT',
                        'Order to supplier','Inner size','Mold invoice issued on','Mold invoice','Stage (from Payments)',
                        'Shipments (from Order items)','Customer Payment #','Customer_y','Stage','Payments_y','Days to payment due',
                        'Status of payment','Status MP downpayment','Num of days','Warning','Invoicing contact (from Customer)_y',
                        'Invoicing address (from Customer)_y','Email (from Invoicing contact) (from Customer)_y',
                        'Today_y','Invoiced vs. paid','Invoice total (from Orders)','% asked (auto)',
                        'Total cost (from Order items) (from Orders)','¥ due to supplier','Supplier payment status',
                        'Paid to supplier','Wire transfer date (July)','Wire transfer amount','Orders_x',
                        'Order # (from Orders)','Name (from Customer) (from Orders)','Manager (from Orders)',
                        'Order items (from Orders)','View invoice (from Orders)','Supplier payment date (July)','Special discount (from Orders)',
           'Status mold payment','Sales Rep','Contacts','Num','Name','Client','Status_y',
            'Lifetime value','Email (from Invoicing contact)','Invoicing address','Email (from Sales Rep)',
            'Count (Addresses) invoice','Inv addresses missing','Count (Addresses) ship','Ship addresses missing',
            'Count contacts','Contact missing','Email (from Contacts)','Update customer info','Orders_y',
            'Total paid (from Orders)','Manager_y','Team Emails (from Managers) (from Teams) (from Manager)_y',
            'Catalogue','Order status (from Orders)','Shipping addresses', 'Shipping address_y','Other names',
           'Payment requests INV','Website','Addresses','Priority','Type (from Addresses)','Products','Invoice Street name and num (from Addresses)',
            'Invoice City and Zipcode (from Addresses)','Invoice Country (from Addresses)',
          'Container Shipping address #2','Accounting contact Name (from Contacts)','Tax ID / DVT / RFC',
           'Artwork contact Name (from Contacts)','Phone','Quotes','Logistics contact Name (from Contacts)',
           'First contact','Last modified time of first contact']
    dfa2.drop(columns=to_drop, inplace=True)
    dfa2.to_csv('orderspaymentscust.csv', index=False)
    return dfa2

In [6]:
def load_data():
    if os.path.exists('orderspaymentscust.csv'):
        df = pd.read_csv('orderspaymentscust.csv')
    else:
        df = clean_fetched()
    
    return df

In [7]:
df = load_data()

In [8]:
# Parameters for retrieving the airtable data
def get_data():
    df = load_data()
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)

    df['Name (from Customer)'] = df['Name (from Customer)'].astype(str).str.replace('[^a-zA-Z0-9]', ' ', regex=True).str.strip()
    df['Num (from Customer)'] = df['Num (from Customer)'].apply(lambda x: int(''.join(filter(str.isdigit, str(x).replace('[', '').replace(']', '')))) if pd.notna(x) else np.nan)
    df['Bottle Type'] = df['Item description (from Order items)'].apply(lambda x: re.findall(r'[a-zA-Z]+', x.split('-')[0])[0] if isinstance(x, str) and '-' in x else 'N/A')
    df['Bundle Bottle'] = df['Item description (from Order items)'].str.extract(r'(\w+ bottle)')
    df['Amount Unpaid'] = df['Total paid'] - df['Order total']
    df['Balance due quartile desc'] = pd.qcut(df['Balance due'], q=4, labels=['3', '2', '1'], duplicates='drop')
    #df['Balance due quartile desc'] = pd.qcut(df['Balance due'], q=4, labels=['4', '3', '2', '1'])
    df['Number of Purchases'] = df.groupby('Num (from Customer)')['Num (from Customer)'].transform('count')
    df['Client Type'] = np.where(df['Number of Purchases'] == 1, 'New', 'Returning')
    df['Average Purchase Amount'] = df.groupby('Num (from Customer)')['Order total'].transform('mean')
    df['Created: project name'] = df['Created: project name'].astype(str)
    df['Clean Date'] = pd.to_datetime(df['Created: project name'].str.replace('nan', ''), format='%Y-%m-%dT%H:%M:%S.%fZ', errors='coerce')
    df['Cost total'] = pd.to_numeric(df['Total cost (from Order items)'], errors='coerce').fillna(0) / 7
    df['Date quote requested'] = pd.to_datetime(df['Date quote requested'])
    df['Date quote created'] = pd.to_datetime(df['Date quote created'])
    df['Date quote sent'] = pd.to_datetime(df['Date quote sent'])
    df['Order date'] = pd.to_datetime(df['Order date'])
    df['Month'] = df['Clean Date'].dt.strftime('%Y-%m')
    df['Sales Rep (from Customer)'] = df['Sales Rep (from Customer)'].astype(str)
    columns_to_keep = ['Date quote accepted', 'Date quote created', 'Date quote requested', 'Date quote sent']
    columns_to_drop = df.columns[df.isnull().mean() >= 0.8].difference(columns_to_keep)
    df.drop(columns=columns_to_drop, inplace=True)

    df = df.sort_values(by=['Num (from Customer)', 'Project name', 'Clean Date'], ascending=[True, True, True])
    
    return df

In [9]:
fetch_table = get_data()

#### Aggs view - 1st layout

In [10]:
#sales per month, per client and per project (total/balance due)
def sales_per_month():
    df = get_data()
    sales_by_month = df.groupby(['Month', 'Name (from Customer)', 'Num (from Customer)', 'Project name']).agg(
        Total_Sales=('Order total', 'sum'),
        Balance_Due=('Balance due', 'sum')
    ).reset_index()   
    return sales_by_month


In [11]:
sales_df = sales_per_month()

In [16]:
#aggregated view per month
def sales_sum_per_month():
    df = sales_per_month()
    sales_sum = df.groupby(['Month'])[['Total_Sales', 'Balance_Due']].sum().reset_index()
    return sales_sum

In [17]:
sales_sum = sales_sum_per_month()

In [18]:
def project_summary():
    df = get_data()
    project_summary = df.groupby(['Name (from Customer)', 'Num (from Customer)']).agg(
        Project_Count=('Project name', 'nunique'),
        Total_Quantity=('Quantity (from Order items)', 'sum'),
        Total_Order=('Order total', 'sum'),
        Total_Cost=('Cost total', 'sum')
    ).reset_index()
    project_summary = project_summary.sort_values(
        by=['Project_Count', 'Total_Order', 'Total_Cost', 'Total_Quantity'],
        ascending=[False, False, False, False]
    )
    
    return project_summary

In [19]:
project_summary = project_summary()

In [20]:
layout={ "barmode": "stack" }

#### Refine selection - 2nd layout

In [21]:
def month_df():
    df = get_data()
    unique_months = sorted(df['Month'].dropna().unique())
    return unique_months

In [22]:
def customer_df():
    df = get_data()
    cust = sorted(df['Name (from Customer)'].unique())
    return cust

In [23]:
def sales_rep():
    df = get_data()
    rep = sorted(df['Sales Rep (from Customer)'].unique())
    return rep  

In [24]:
month_lov = month_df()
cust_lov = customer_df() 
rep_lov = sales_rep()

selected_month = month_lov[0]
selected_cust = cust_lov[0] 
selected_rep = rep_lov[0]

In [25]:
def sales_sum_per_month(selected_month):
    df = sales_per_month()  # You may need to pass the selected_month to the sales_per_month function as well
    df = df[df['Month'] == selected_month]
    sales_sum = df.groupby(['Month'])[['Total_Sales', 'Balance_Due']].sum().reset_index()
    return sales_sum


In [26]:
sales_on_month = sales_sum_per_month(selected_month)['Total_Sales']
balance_on_month = sales_sum_per_month(selected_month)['Balance_Due']

In [27]:
def fetch_project_details(month, cust):
    df = get_data()
    df = df[df['Name (from Customer)'] == cust]

    #filter for month
    if isinstance(month, str):
        month = [month]
    if month:
        df = df[df['Month'].isin(month)]

    project_details = df.groupby(['Project name']).agg(
        Total_Sales=('Order total', 'sum'),
        Balance_Due=('Balance due', 'sum')
    ).reset_index()
    
    return project_details


In [28]:
project_details = fetch_project_details(selected_month, selected_cust)

In [29]:
def gantt_df(month,rep):
    df = get_data()
    filtered_df = df[(df['Sales Rep (from Customer)'] == sales_rep) & (df['Month'] == month)]

    # Convert date columns to datetime objects
    date_columns = ['Order date', 'Date quote created', 'Date quote accepted']
    for col in date_columns:
        filtered_df[col] = pd.to_datetime(filtered_df[col])
    
    # Group by 'Order num' and 'Name (from Customer)'
    grouped = filtered_df.groupby(['Name (from Customer)','Project name'])
    
    # Create the Gantt chart data
    gantt_data = []
    
    for (order_num, customer), group in grouped:
        order_dates = group['Order date'].values[0]
        for _, row in group.iterrows():
            gantt_data.append([customer, order_num, row['Quote status'], order_dates, row['Date quote created'], row['Date quote accepted']])
            order_dates = row['Date quote accepted']
    
    # Create a DataFrame for the Gantt chart data
    gantt_df = pd.DataFrame(gantt_data, columns=['Customer', 'Project name', 'Quote status', 'Start', 'Create', 'Accepted'])
    return gantt_df


In [30]:
gantt_rep = gantt_df(selected_month,selected_rep)

#### Segmentation heatmap

In [31]:
def create_rfm_table():
    df = get_data()
    NOW = df['Clean Date'].max() + timedelta(days=1)
    period = 365
    df['Days since last order'] = df['Clean Date'].apply(lambda x: (NOW - x).days)
    aggr = {
        'Days since last order': lambda x: x.min(),  # the number of days since last order (Recency)
        'Clean Date': lambda x: len([d for d in x if d >= NOW - timedelta(days=period)]), # the total number of orders in the last period (Frequency)
    }
    rfm = df.groupby('Num (from Customer)').agg(aggr).reset_index()
    rfm.rename(columns={'Days since last order': 'Recency', 'Clean Date': 'Frequency'}, inplace=True)
    rfm['Monetary'] = rfm['Num (from Customer)'].apply(lambda x: df[(df['Num (from Customer)'] == x) & \
                                                               (df['Clean Date'] >= NOW - timedelta(days=period))]\
                                                               ['Order total'].sum())
    quintiles = rfm[['Recency', 'Frequency', 'Monetary']].quantile([.2, .4, .6, .8]).to_dict()
    #assign ranks whereby higher monet and higher freq are better but recency smaller better
    def r_score(x):
        if x <= quintiles['Recency'][.2]:
            return 5
        elif x <= quintiles['Recency'][.4]:
            return 4
        elif x <= quintiles['Recency'][.6]:
            return 3
        elif x <= quintiles['Recency'][.8]:
            return 2
        else:
            return 1
    
    def fm_score(x, c):
        if x <= quintiles[c][.2]:
            return 1
        elif x <= quintiles[c][.4]:
            return 2
        elif x <= quintiles[c][.6]:
            return 3
        elif x <= quintiles[c][.8]:
            return 4
        else:
            return 5    
    #apply for each cust
    rfm['R'] = rfm['Recency'].apply(lambda x: r_score(x))
    rfm['F'] = rfm['Frequency'].apply(lambda x: fm_score(x, 'Frequency'))
    rfm['M'] = rfm['Monetary'].apply(lambda x: fm_score(x, 'Monetary'))
    rfm['RFM Score'] = rfm['R'].map(str) + rfm['F'].map(str) + rfm['M'].map(str)
    # Function to map RFM scores to segments
    rfm['RFM Score'] = rfm['RFM Score'].astype(str).astype(int)
    def map_segment(rfm_score):
        if rfm_score in [111, 112, 121, 131, 141, 151]:
            return 'Lost'
        elif rfm_score in [332, 322, 231, 241, 251, 233, 232, 223, 222, 132, 123, 122, 212, 211]:
            return 'Hibernating'
        elif rfm_score in [255, 254, 245, 244, 253, 252, 243, 242, 235, 234, 225, 224, 153, 152, 145, 143, 142, 135, 134, 133, 125, 124]:
            return 'At Risk'
        elif rfm_score in [331, 321, 312, 221, 213]:
            return 'About To Sleep'
        elif rfm_score in [155, 154, 144, 214, 215, 115, 114, 113]:
            return 'Cannot Lose Them'
        elif rfm_score in [535, 534, 443, 434, 343, 334, 325, 324]:
            return 'Need Attention'
        elif rfm_score in [525, 524, 523, 522, 521, 515, 514, 513, 425, 424, 413, 414, 415, 315, 314, 313]:
            return 'Promising'
        elif rfm_score in [512, 511, 422, 421, 412, 411, 311]:
            return 'New Customer'
        elif rfm_score in [553, 551, 552, 541, 542, 533, 532, 531, 452, 451, 442, 441, 431, 453, 433, 432, 423, 353, 352, 351, 342, 341, 333, 323]:
            return 'Potential Loyalist'
        elif rfm_score in [543, 444, 435, 355, 354, 345, 344, 335]:
            return 'Loyal Customer'
        elif rfm_score in [555, 554, 544, 545, 454, 455, 445]:
            return 'Champion'
        else:
            return 'Unknown'
    
    # Apply the mapping function to create the 'Segment' column
    rfm['Segment'] = rfm['RFM Score'].apply(map_segment)
    # Dictionary to map segments to behavior and potential strategy
    segment_mapping = {
        'Champion': {
            'Segment Behavior': 'Completed a recent purchase. They buy frequently and spend the most. Reward them. Can be early adopters of new products. They act as brand ambassadors.',
            'Segment Potential Strategy': 'Reward them. Can be early adopters of new products. They act as brand ambassadors.'
        },
        'Loyal Customer': {
            'Segment Behavior': 'Buy on a regular basis. Responsive to promotions. Upsell. Offer high-value items to them. Ask them for reviews. Build loyalty.',
            'Segment Potential Strategy': 'Upsell. Offer high-value items to them. Ask them for reviews. Build loyalty.'
        },
        'Potential Loyalist': {
            'Segment Behavior': 'They recently spent a fair amount of money on the site more than once. Invite them to join a loyalty program or to get a membership. Recommend other products to them.',
            'Segment Potential Strategy': 'Invite them to join a loyalty program or to get a membership. Recommend other products to them.'
        },
        'Promising': {
            'Segment Behavior': 'Bought most recently, but not often enough for us to understand their behavior. Support them on their onboarding process. Get their satisfaction. Start to build loyalty.',
            'Segment Potential Strategy': 'Support them on their onboarding process. Get their satisfaction. Start to build loyalty.'
        },
        'New Customer': {
            'Segment Behavior': 'Completed a recent purchase, but didn’t spend a lot. Create brand awareness. Offer discounts, gifts, and free trials.',
            'Segment Potential Strategy': 'Create brand awareness. Offer discounts, gifts, and free trials.'
        },
        'Need Attention': {
            'Segment Behavior': 'Above average in recency, frequency, and monetary value, but no recent purchases. Launch limited-time offers and recommendations based on previous orders. Reactivate.',
            'Segment Potential Strategy': 'Launch limited-time offers and recommendations based on previous orders. Reactivate.'
        },
        'Cannot Lose Them': {
            'Segment Behavior': 'Completed huge, frequent purchases long ago but haven’t come back since then. Get them back with new product launches. Don’t let competitors take them. Talk to them.',
            'Segment Potential Strategy': 'Get them back with new product launches. Don’t let competitors take them. Talk to them.'
        },
        'About To Sleep': {
            'Segment Behavior': 'Below average in recency, frequency, and monetary value. At risk of losing them if you don’t reactivate. Share useful information. Recommend popular products to them. Offer new discounts. Reconnect.',
            'Segment Potential Strategy': 'Share useful information. Recommend popular products to them. Offer new discounts. Reconnect.'
        },
        'At Risk': {
            'Segment Behavior': 'Purchased often but a long time ago. Need to bring them back! Send personalized emails to reconnect with special offers. Share useful resources with them.',
            'Segment Potential Strategy': 'Send personalized emails to reconnect with special offers. Share useful resources with them.'
        },
        'Hibernating': {
            'Segment Behavior': 'Their last purchase was long ago. The number of orders was small and the amounts spent low. Offer relevant products and special discounts. Create brand value again.',
            'Segment Potential Strategy': 'Offer relevant products and special discounts. Create brand value again.'
        },
        'Lost': {
            'Segment Behavior': 'Lowest scores in recency, frequency, and monetary value. Try to reactivate them with personalized campaigns. If it doesn’t work, ignore them.',
            'Segment Potential Strategy': 'Try to reactivate them with personalized campaigns. If it doesn’t work, ignore them.'
        }
    }
    
    # Map the segments to behavior and potential strategy
    rfm['Segment Behavior'] = rfm['Segment'].map(lambda x: segment_mapping[x]['Segment Behavior'])
    rfm['Segment Potential Strategy'] = rfm['Segment'].map(lambda x: segment_mapping[x]['Segment Potential Strategy'])
    return rfm

In [32]:
def rfm_heatmap():
    rfm = create_rfm_table()
    rfm_avg = rfm.groupby('Segment').agg({
        'R': lambda x: x.mode().iloc[0],
        'F': lambda x: x.mode().iloc[0],
        'M': lambda x: x.mode().iloc[0],
        'Num (from Customer)': 'count'
    }).round(1)
    rfm_avg.rename(columns={'Num (from Customer)': 'Count'}, inplace=True)
    return rfm_avg.reset_index()

In [33]:
def client_segment_map(selected_cust):
    rfm = create_rfm_table()
    df = get_data()
    df = df.merge(rfm, on='Num (from Customer)', how='left')
    return df[df['Name (from Customer)'].str.contains(selected_cust)][['Name (from Customer)', 'Segment','Segment Behavior','Segment Potential Strategy']].head(1)

In [34]:
segment_map = client_segment_map(selected_cust)

#### Page - markdown

In [35]:
# dv_graph_selector = ['Sales Dashboard','Client Segmentation','Cohort Analysis']
# dv_graph_selected = dv_graph_selector[0]

In [36]:
# Root page - page shared by everyone
root_md = """
<center><h2> Rock **KYC** </h2></center> <right><|Refresh Airtable|button|on_action=refresh_table|></right>

--------------------------------------------------------------------
<center><|navbar|></center>
"""

In [37]:
# Sales dashboard page
sales_dashboard_md = """
<h4> Sales Dashboard (CNY) </h4>

<|layout|columns=1 1|
<|{sales_sum}|chart|mode=lines|x=Month|y[1]=Total_Sales|y[2]=Balance_Due|line[2]=dash|color[1]=blue|>

<|{project_summary}|chart|type=bar|x=Name (from Customer)|y[1]=Total_Order|y[2]=Total_Cost|layout={layout}|>
|>

<|project_summary|expandable|
    ...
<|{project_summary}|table|>
    ...
|>

---------------------
Select Month <|{selected_cust}|selector|lov={cust_lov}|dropdown|on_change=update_table|>  Customer <|{selected_month}|selector|lov={month_lov}|dropdown|on_change=update_table|> Rep  <|{selected_rep}|selector|lov={rep_lov}|dropdown|on_change=update_table|>
<|layout|columns=1 1|
For month <|{selected_month}|>   Total Orders =  <|{sales_on_month}|> Total Balance due <|{balance_on_month}|>
<|{project_details}|table|show_all|> <|{gantt_rep}|table|show_all|>
|>
"""

In [38]:
# Client segmentation page
client_segmentation_md = """
<h4> Client segmentation by behaviour & transaction </h4>

<|layout|columns = 1 5|
<h6>Search for a client (enter name):  </h6>
<|{selected_cust}|selector|lov={cust_lov}|dropdown|on_change=update_table|>
<|{segment_map}|table|show_all|>

|>
"""


In [39]:
# Cohort analysis page
cohort_analysis_md = """
<h4> Client cohort (behaviour evolution) analysis  </h4>
"""


In [40]:
def refresh_table(state):
    state.fetch_airtable = get_data()

In [41]:
def refresh_data(state):
    state.sales_df = sales_per_month()
    state.contributors = project_summary()
    state.month_lov = month_df()
    state.cust_lov = customer_df()
    state.selected_month = state.month_lov[0]
    state.selected_cust = state.cust_lov[0]
    state.selected_rep = state.rep_lov[0]
    state.project_details = fetch_project_details(state.selected_month, state.selected_cust)
    state.gantt_rep = gantt_df(state.selected_month, state.selected_rep)
    state.segment_map = client_segment_map(state.selected_cust)


In [42]:
def update_table(state):
    state.project_details = fetch_project_details(state.selected_month, state.selected_cust)
    state.gantt_rep = gantt_df(state.selected_month, state.selected_rep)
    state.segment_map = client_segment_map(state.selected_cust)

In [43]:
# Putting pages in a dict
pages = {
    '/': root_md,
    'Sales-Dashboard': sales_dashboard_md,
    'Client-Segmentation': client_segmentation_md,
    'Cohort-Analysis': cohort_analysis_md
}


In [44]:
# Run Taipy GUI
gui = Gui(pages=pages)
gui.run(dark_mode=False)
     

[2023-10-12 17:46:40][Taipy][INFO] Running in 'single_client' mode in notebook environment
[2023-10-12 17:46:43][Taipy][INFO]  * Server starting on http://127.0.0.1:5000


--- 1 warning(s) were found for page 'Sales-Dashboard' in variable 'sales_dashboard_md' ---
 - Warning 1: Exception in JSONEncoder:
Object of type Series is not JSON serializable
--------------------------------------------------------------------------------------------

--- 1 warning(s) were found for page 'Sales-Dashboard' in variable 'sales_dashboard_md' ---
 - Warning 1: Exception in JSONEncoder:
Object of type Series is not JSON serializable
--------------------------------------------------------------------------------------------

--- 1 warning(s) were found for page 'Sales-Dashboard' in variable 'sales_dashboard_md' ---
 - Warning 1: Exception in JSONEncoder:
Object of type Series is not JSON serializable
--------------------------------------------------------------------------------------------

Invalid session ADJcc-mcjhqCH0tTAAAA (further occurrences of this error will be logged with level INFO)
ERROR:engineio.server:Invalid session ADJcc-mcjhqCH0tTAAAA (further occurrenc

# STOP

#### Next page on client segmentation

In [ ]:
get_data().head(2)

In [ ]:
def create_rfm_table():
    df = get_data()
    NOW = df['Clean Date'].max() + timedelta(days=1)
    period = 365
    df['Days since last order'] = df['Clean Date'].apply(lambda x: (NOW - x).days)
    aggr = {
        'Days since last order': lambda x: x.min(),  # the number of days since last order (Recency)
        'Clean Date': lambda x: len([d for d in x if d >= NOW - timedelta(days=period)]), # the total number of orders in the last period (Frequency)
    }
    rfm = df.groupby('Num (from Customer)').agg(aggr).reset_index()
    rfm.rename(columns={'Days since last order': 'Recency', 'Clean Date': 'Frequency'}, inplace=True)
    rfm['Monetary'] = rfm['Num (from Customer)'].apply(lambda x: df[(df['Num (from Customer)'] == x) & \
                                                               (df['Clean Date'] >= NOW - timedelta(days=period))]\
                                                               ['Order total'].sum())
    quintiles = rfm[['Recency', 'Frequency', 'Monetary']].quantile([.2, .4, .6, .8]).to_dict()
    #assign ranks whereby higher monet and higher freq are better but recency smaller better
    def r_score(x):
        if x <= quintiles['Recency'][.2]:
            return 5
        elif x <= quintiles['Recency'][.4]:
            return 4
        elif x <= quintiles['Recency'][.6]:
            return 3
        elif x <= quintiles['Recency'][.8]:
            return 2
        else:
            return 1
    
    def fm_score(x, c):
        if x <= quintiles[c][.2]:
            return 1
        elif x <= quintiles[c][.4]:
            return 2
        elif x <= quintiles[c][.6]:
            return 3
        elif x <= quintiles[c][.8]:
            return 4
        else:
            return 5    
    #apply for each cust
    rfm['R'] = rfm['Recency'].apply(lambda x: r_score(x))
    rfm['F'] = rfm['Frequency'].apply(lambda x: fm_score(x, 'Frequency'))
    rfm['M'] = rfm['Monetary'].apply(lambda x: fm_score(x, 'Monetary'))
    rfm['RFM Score'] = rfm['R'].map(str) + rfm['F'].map(str) + rfm['M'].map(str)
    # Function to map RFM scores to segments
    rfm['RFM Score'] = rfm['RFM Score'].astype(str).astype(int)
    def map_segment(rfm_score):
        if rfm_score in [111, 112, 121, 131, 141, 151]:
            return 'Lost'
        elif rfm_score in [332, 322, 231, 241, 251, 233, 232, 223, 222, 132, 123, 122, 212, 211]:
            return 'Hibernating'
        elif rfm_score in [255, 254, 245, 244, 253, 252, 243, 242, 235, 234, 225, 224, 153, 152, 145, 143, 142, 135, 134, 133, 125, 124]:
            return 'At Risk'
        elif rfm_score in [331, 321, 312, 221, 213]:
            return 'About To Sleep'
        elif rfm_score in [155, 154, 144, 214, 215, 115, 114, 113]:
            return 'Cannot Lose Them'
        elif rfm_score in [535, 534, 443, 434, 343, 334, 325, 324]:
            return 'Need Attention'
        elif rfm_score in [525, 524, 523, 522, 521, 515, 514, 513, 425, 424, 413, 414, 415, 315, 314, 313]:
            return 'Promising'
        elif rfm_score in [512, 511, 422, 421, 412, 411, 311]:
            return 'New Customer'
        elif rfm_score in [553, 551, 552, 541, 542, 533, 532, 531, 452, 451, 442, 441, 431, 453, 433, 432, 423, 353, 352, 351, 342, 341, 333, 323]:
            return 'Potential Loyalist'
        elif rfm_score in [543, 444, 435, 355, 354, 345, 344, 335]:
            return 'Loyal Customer'
        elif rfm_score in [555, 554, 544, 545, 454, 455, 445]:
            return 'Champion'
        else:
            return 'Unknown'
    
    # Apply the mapping function to create the 'Segment' column
    rfm['Segment'] = rfm['RFM Score'].apply(map_segment)
    # Dictionary to map segments to behavior and potential strategy
    segment_mapping = {
        'Champion': {
            'Segment Behavior': 'Completed a recent purchase. They buy frequently and spend the most. Reward them. Can be early adopters of new products. They act as brand ambassadors.',
            'Segment Potential Strategy': 'Reward them. Can be early adopters of new products. They act as brand ambassadors.'
        },
        'Loyal Customer': {
            'Segment Behavior': 'Buy on a regular basis. Responsive to promotions. Upsell. Offer high-value items to them. Ask them for reviews. Build loyalty.',
            'Segment Potential Strategy': 'Upsell. Offer high-value items to them. Ask them for reviews. Build loyalty.'
        },
        'Potential Loyalist': {
            'Segment Behavior': 'They recently spent a fair amount of money on the site more than once. Invite them to join a loyalty program or to get a membership. Recommend other products to them.',
            'Segment Potential Strategy': 'Invite them to join a loyalty program or to get a membership. Recommend other products to them.'
        },
        'Promising': {
            'Segment Behavior': 'Bought most recently, but not often enough for us to understand their behavior. Support them on their onboarding process. Get their satisfaction. Start to build loyalty.',
            'Segment Potential Strategy': 'Support them on their onboarding process. Get their satisfaction. Start to build loyalty.'
        },
        'New Customer': {
            'Segment Behavior': 'Completed a recent purchase, but didn’t spend a lot. Create brand awareness. Offer discounts, gifts, and free trials.',
            'Segment Potential Strategy': 'Create brand awareness. Offer discounts, gifts, and free trials.'
        },
        'Need Attention': {
            'Segment Behavior': 'Above average in recency, frequency, and monetary value, but no recent purchases. Launch limited-time offers and recommendations based on previous orders. Reactivate.',
            'Segment Potential Strategy': 'Launch limited-time offers and recommendations based on previous orders. Reactivate.'
        },
        'Cannot Lose Them': {
            'Segment Behavior': 'Completed huge, frequent purchases long ago but haven’t come back since then. Get them back with new product launches. Don’t let competitors take them. Talk to them.',
            'Segment Potential Strategy': 'Get them back with new product launches. Don’t let competitors take them. Talk to them.'
        },
        'About To Sleep': {
            'Segment Behavior': 'Below average in recency, frequency, and monetary value. At risk of losing them if you don’t reactivate. Share useful information. Recommend popular products to them. Offer new discounts. Reconnect.',
            'Segment Potential Strategy': 'Share useful information. Recommend popular products to them. Offer new discounts. Reconnect.'
        },
        'At Risk': {
            'Segment Behavior': 'Purchased often but a long time ago. Need to bring them back! Send personalized emails to reconnect with special offers. Share useful resources with them.',
            'Segment Potential Strategy': 'Send personalized emails to reconnect with special offers. Share useful resources with them.'
        },
        'Hibernating': {
            'Segment Behavior': 'Their last purchase was long ago. The number of orders was small and the amounts spent low. Offer relevant products and special discounts. Create brand value again.',
            'Segment Potential Strategy': 'Offer relevant products and special discounts. Create brand value again.'
        },
        'Lost': {
            'Segment Behavior': 'Lowest scores in recency, frequency, and monetary value. Try to reactivate them with personalized campaigns. If it doesn’t work, ignore them.',
            'Segment Potential Strategy': 'Try to reactivate them with personalized campaigns. If it doesn’t work, ignore them.'
        }
    }
    
    # Map the segments to behavior and potential strategy
    rfm['Segment Behavior'] = rfm['Segment'].map(lambda x: segment_mapping[x]['Segment Behavior'])
    rfm['Segment Potential Strategy'] = rfm['Segment'].map(lambda x: segment_mapping[x]['Segment Potential Strategy'])
    return rfm

In [ ]:
create_rfm_table().head(2)

In [ ]:
def rfm_heatmap():
    rfm = create_rfm_table()
    # Calculate average values for each RFM_Level, and return a size of each segment 
    rfm_avg = rfm.groupby('Segment').agg({
        'R': lambda x: x.mode().iloc[0],
        'F': lambda x: x.mode().iloc[0],
        'M': lambda x: x.mode().iloc[0],
        'Num (from Customer)': 'count'
    }).round(1)
    rfm_avg.rename(columns={'Num (from Customer)': 'Count'}, inplace=True)
    # # Create our plot and resize it.
    # fig, ax = plt.subplots(figsize=(16, 9))
    
    # # Use squarify to create the treemap
    # sizes = rfm_avg['Count']
    # labels = [f'{label} ({count})' for label, count in zip(rfm_avg.index, sizes)]
    # #colors = plt.cm.Paired(range(len(sizes)))  
    # colors = sns.color_palette("coolwarm", len(sizes))
    # #colors = sns.diverging_palette(250, 30, l=65, center="silver",len(sizes))

    # squarify.plot(sizes=sizes, label=labels, alpha=0.6, color=colors)
    
    # # Add title and axis labels
    # plt.title("RFM Segments")
    
    # return plt.show()
    return rfm_avg.reset_index()

In [ ]:
rfm_heatmap()

In [ ]:
# def rfm_bars():
#     rfm = create_rfm_table()
#     # count the number of customers in each segment
#     segments_counts = rfm['Segment'].value_counts().sort_values(ascending=True)
    
#     fig, ax = plt.subplots()
    
#     bars = ax.barh(range(len(segments_counts)),
#                   segments_counts,
#                   color='silver')
#     ax.set_frame_on(False)
#     ax.tick_params(left=False,
#                    bottom=False,
#                    labelbottom=False)
#     ax.set_yticks(range(len(segments_counts)))
#     ax.set_yticklabels(segments_counts.index)
    
#     for i, bar in enumerate(bars):
#             value = bar.get_width()
#             if segments_counts.index[i] in ['Champion', 'Loyal Customer']:
#                 bar.set_color('firebrick')
#             ax.text(value,
#                     bar.get_y() + bar.get_height()/2,
#                     '{:,} ({:}%)'.format(int(value),
#                                        int(value*100/segments_counts.sum())),
#                     va='center',
#                     ha='left'
#                    )
    
#     return plt.show()

In [ ]:
def rfm_bars():
    rfm = create_rfm_table()
    # count the number of customers in each segment
    segments_counts = rfm['Segment'].value_counts().sort_values(ascending=True)
    return segments_counts
    # fig, ax = plt.subplots()
    
    # bars = ax.barh(range(len(segments_counts)),
    #               segments_counts,
    #               color='silver')
    # ax.set_frame_on(False)
    # ax.tick_params(left=False,
    #                bottom=False,
    #                labelbottom=False)
    # ax.set_yticks(range(len(segments_counts)))
    # ax.set_yticklabels(segments_counts.index)
    
    # for i, bar in enumerate(bars):
    #         value = bar.get_width()
    #         if segments_counts.index[i] in ['Champion', 'Loyal Customer']:
    #             bar.set_color('firebrick')
    #         ax.text(value,
    #                 bar.get_y() + bar.get_height()/2,
    #                 '{:,} ({:}%)'.format(int(value),
    #                                    int(value*100/segments_counts.sum())),
    #                 va='center',
    #                 ha='left'
    #                )
    
    # return plt.show()

In [ ]:
# properties = {
#     segments_counts,
#     #colors=['silver' if idx not in ['Champion', 'Loyal Customer'] else 'firebrick' for idx in segments_counts.index],
#     "y":segments_counts.index,
#     "x":range(len(segments_counts),
#     "orientation":    "h",
#     "layout": {
#         # Set a relevant title for the x axis
#         "xaxis": { "title": "Counts" },
#         # Hide the legend
#         "showlegend": True
# }

In [ ]:
rfm_bars()

In [ ]:
search_client = 'WILLIBALD'

In [ ]:
def client_segment_map(search_client):
    rfm = create_rfm_table()
    df = get_data()
    df = df.merge(rfm, on='Num (from Customer)', how='left')
    return df[df['Name (from Customer)'].str.contains(search_client)][['Name (from Customer)', 'Segment','Segment Behavior','Segment Potential Strategy']]

In [ ]:
client_segment_map(search_client)

In [ ]:
# dv_graph_selector = ['Client Segmentation','Cohort Analysis']
# dv_graph_selected = dv_graph_selector[0]

In [ ]:
page =  """
<center><h2> Rock **KYC** </h2></center>

<|{dv_graph_selected}|toggle|lov={dv_graph_selector}|>

--------------------------------------------------------------------
<|part|render={dv_graph_selected == 'Client Segmentation'}|
<h4> Client segmentation by behaviour & transaction </h4>

<|layout|columns = 1 5|
<|
<h6>Search for a client (enter name):  </h6>
<|{search_client}|input|> 
|>

<|
<h6><|{search_client}|> summary and action: </h6>
<|{client_segment_map(search_client)}|table|width=80%|>
|>
|>

same row 2 columns for rfm_heatmap() rfm_bars()
<|layout|columns = 1 1|
<|
<h6>Our clients overall </h6>
<|{client_segment_map(search_client)}|chart|type=heatmap|z=Count|x=R|y=F|layout={layout}|>
|>
<| 
<h6>Our clients by size </h6>
<|{rfm_bars()}|chart|type=bar|>
|>
|>

|>

<|part|render={dv_graph_selected == 'Cohort Analysis'}|
<h4> Client cohort (behaviour evolution) analysis  </h4>



|>
"""

In [ ]:
# def on_change():
    

In [ ]:
# # Run Taipy GUI
# gui = Gui(page)
# gui.run(dark_mode=False)
     